In [1]:
from pathlib import Path

In [2]:
import re

In [3]:
#!head -150 IDMS_RECORDS_cpy.txt | tail -50

In [4]:
import pandas as pd

In [5]:
#ls

In [6]:
pd.set_option( 'display.max_rows', 200 )

In [7]:
#subsystems = pd.read_excel( "PLAS Subsystem Schema -Revised.xlsx", index_col=0)

# PLAS subsystems names

In [8]:
subsystems = pd.read_excel( "PLAS Subsystem Schema -Revised.xlsx", index_col=0, sheet_name=1)

In [9]:
wanted_columns = ['Program\nSchema', 'System Name' ]

In [10]:
drop_these = set( subsystems.columns ) - set( wanted_columns )

In [11]:
subsystems = subsystems.drop( columns=drop_these )

In [12]:
subsystems.columns = ['code', 'plas_subsystem' ]

In [13]:
subsystems[ subsystems['code'] == 'BD' ]

,code,plas_subsystem
#,,
22,BD,This subsystem performs maintenance on the App...
122,BD,INITIALIZE APPROPRIATION ACCOUNTING FILES


In [14]:
subsystems = subsystems.drop( index = 122 )

## Systems without a code

In [15]:
no_code = subsystems[ subsystems['code'] == '**' ]

In [16]:
no_code

,code,plas_subsystem
#,,
31,**,NOTICE OF MATURITY
56,**,MERGE PROCESS
60,**,REPORT CODE 523 - GENERAL LEDGER TRIAL BALANCE...
160,**,SA-SK AUTOMATED MULTI-HOUSING ACCOUNTING SYSTE...
162,**,Budget and Appropriation
163,**,Online Cash Payment Processing
164,**,Monthly Billings
165,**,Tenant File
166,**,Loan Making/Receivable Processing


In [17]:
subsystems = subsystems.drop( index = no_code.index )

In [18]:
subsystems.shape

(181, 2)

In [19]:
subsystems = subsystems.dropna( subset=['code'] )

In [20]:
subsystems.shape

(180, 2)

In [21]:
subsystems.sort_values( 'code' )

,code,plas_subsystem
#,,
1,AA,INITIALIZE FILES
2,AB,BLOCKLISTING PROCESSING
46,AD,DISCREPANCY PROCESSING AND ACCOUNT INFORMATION...
161,ADPS,Online Transaction and Discrepancy Processing
3,AE,BORROWER CHECK PROCESSING
4,AF,HOLDER CHECK PROCESSING
5,AG,INPUT-TO-OUTPUT BALANCING
6,AH,PRELIMINARY REGISTER ACTIVITY
7,AI,PROCESSED AND DISCREPANCY REGISTERS


In [22]:
two_letter_code_to_subsystem_desc_map = dict( subsystems.values )

In [23]:
#!cp /Users/ccoletta/Library/CloudStorage/Box-Box/GSA\ USDA\ -\ CONTRACTOR\ ACCESS/PLAS_CODE_AND_DOCUMENTAION/file_lengths.txt .

# Analysis of COBOL programs obtained in discovery (line counts for each file)

In [24]:
files = pd.read_csv( 'file_lengths.txt', header=None  )

In [25]:
files.columns = ['raw_line']

In [26]:
files = files['raw_line'].str.split(expand=True)

In [27]:
files.columns = ['line_count', 'file_path']

In [28]:
files['n_slashes'] = files['file_path'].str.count('/')

In [29]:
files['n_slashes'].value_counts()

2    14830
Name: n_slashes, dtype: int64

In [30]:
#files['file_path'].str.contains( 'PLAS_documentation' ).value_counts()
#files = files[ ~files['file_path'].str.contains( 'PLAS_documentation' ) ].copy()
#files = files[ ~files['file_path'].str.contains( 'archive_of_archives' ) ].copy()
#files = files[ files['n_slashes'] == 2].copy()

In [31]:
path_parts = files['file_path'].str.split( '/', expand=True )

In [32]:
files[ ['file_dir', 'file_name' ] ] = path_parts[ [1,2] ]

In [33]:
files

,line_count,file_path,n_slashes,file_dir,file_name
0,2189,./ASL900.SORCLIB.CBL/KSBR02,2,ASL900.SORCLIB.CBL,KSBR02
1,733,./ASL900.SORCLIB.CBL/PHBS7A,2,ASL900.SORCLIB.CBL,PHBS7A
2,439,./ASL900.SORCLIB.CBL/SEMDD036,2,ASL900.SORCLIB.CBL,SEMDD036
3,1468,./ASL900.SORCLIB.CBL/AIBS16,2,ASL900.SORCLIB.CBL,AIBS16
4,1260,./ASL900.SORCLIB.CBL/BUBR53,2,ASL900.SORCLIB.CBL,BUBR53
...,...,...,...,...,...
14825,247,./MISSING.SORCLIB.TXT/ETBS01,2,MISSING.SORCLIB.TXT,ETBS01
14826,482,./MISSING.SORCLIB.TXT/ATBS03,2,MISSING.SORCLIB.TXT,ATBS03
14827,12745,./MISSING.SORCLIB.TXT/TAXDUMP,2,MISSING.SORCLIB.TXT,TAXDUMP
14828,266,./MISSING.SORCLIB.TXT/HNBANS,2,MISSING.SORCLIB.TXT,HNBANS


In [34]:
files = files[ ['line_count', 'file_dir', 'file_name' ] ].copy()

In [35]:
files = files.reset_index(drop=True)

In [36]:
files['file_name_len'] = files['file_name'].apply( len ) 

In [37]:
files['file_name_len'].value_counts()

8    8177
6    6526
7     115
4       7
5       5
Name: file_name_len, dtype: int64

In [38]:
#tuple( two_letter_code_to_subsystem_desc_map.keys() )

In [39]:
files.shape

(14830, 4)

In [40]:
files = files.sort_values( ['file_dir', 'file_name' ] )

In [41]:
files

,line_count,file_dir,file_name,file_name_len
8318,4,ASL900.PROCLIB,AADCARD,7
6727,31,ASL900.PROCLIB,AAPABGEN,8
7330,18,ASL900.PROCLIB,AAPABM08,8
6630,14,ASL900.PROCLIB,AAPABNDC,8
7290,19,ASL900.PROCLIB,AAPABR15,8
...,...,...,...,...
6144,639,SRCLIB.CBL,YBBR08,6
5345,1359,SRCLIB.CBL,YDBR02,6
5479,419,SRCLIB.CBL,YDBR03,6
5498,1117,SRCLIB.CBL,YDBR04,6


In [42]:
file_counts = files['file_name'].value_counts()

In [43]:
dupes = set( file_counts[ file_counts != 1 ].index )

In [44]:
multiple_places = files[ files['file_name'].isin( dupes ) ].groupby( 'file_name' )[ 'file_dir' ].agg( set )

In [45]:
multiple_places['ABBM08']

{'ASL900.SORCLIB.CBL', 'LISTLIB.TXT', 'SORCLIB.TXT1', 'SRCLIB.CBL'}

## Match COBOL filenames to their PLAS subsystem via two-letter naming convention

In [46]:
valid_codes = tuple( sorted( list( two_letter_code_to_subsystem_desc_map.keys() ) ) )

In [47]:
files['categorized'] = files['file_name'].str.startswith( valid_codes )

In [48]:
files['categorized'].value_counts()

True     8141
False    6689
Name: categorized, dtype: int64

In [49]:
files

,line_count,file_dir,file_name,file_name_len,categorized
8318,4,ASL900.PROCLIB,AADCARD,7,True
6727,31,ASL900.PROCLIB,AAPABGEN,8,True
7330,18,ASL900.PROCLIB,AAPABM08,8,True
6630,14,ASL900.PROCLIB,AAPABNDC,8,True
7290,19,ASL900.PROCLIB,AAPABR15,8,True
...,...,...,...,...,...
6144,639,SRCLIB.CBL,YBBR08,6,False
5345,1359,SRCLIB.CBL,YDBR02,6,False
5479,419,SRCLIB.CBL,YDBR03,6,False
5498,1117,SRCLIB.CBL,YDBR04,6,False


In [50]:
from collections import Counter

In [51]:
categorized = files[ files['categorized'] ]

In [52]:
Counter( [ _[:2] for _ in categorized['file_name'].values ] ).most_common(n=30)

[('AA', 2100),
 ('AD', 749),
 ('JE', 344),
 ('PD', 273),
 ('AU', 266),
 ('TS', 263),
 ('DQ', 262),
 ('JN', 194),
 ('CR', 168),
 ('BU', 150),
 ('KJ', 137),
 ('AE', 127),
 ('MU', 112),
 ('AW', 107),
 ('JV', 104),
 ('EC', 103),
 ('LN', 98),
 ('JU', 96),
 ('LQ', 94),
 ('GA', 90),
 ('HE', 82),
 ('PE', 81),
 ('BS', 76),
 ('GD', 72),
 ('KN', 72),
 ('BA', 64),
 ('NW', 63),
 ('OB', 62),
 ('MX', 61),
 ('AB', 59)]

In [53]:
Counter( [ _[:3] for _ in categorized['file_name'].values ] ).most_common(n=30)

[('AAP', 2099),
 ('JEB', 333),
 ('ADT', 332),
 ('DQB', 262),
 ('AUB', 257),
 ('ADB', 244),
 ('JNB', 194),
 ('PDB', 171),
 ('CRB', 160),
 ('ADP', 159),
 ('BUB', 146),
 ('TSB', 142),
 ('KJB', 131),
 ('AEB', 122),
 ('MUB', 112),
 ('JVB', 104),
 ('TSM', 98),
 ('JUB', 96),
 ('PDG', 94),
 ('ECB', 94),
 ('LQB', 92),
 ('AWB', 91),
 ('GAB', 90),
 ('LNB', 88),
 ('HEB', 80),
 ('BSB', 74),
 ('KNB', 72),
 ('GDB', 70),
 ('PEB', 67),
 ('NWB', 63)]

In [54]:
Counter( [ _[:4] for _ in categorized['file_name'].values ] ).most_common(n=30)

[('AAPS', 420),
 ('DQBS', 246),
 ('AAPJ', 224),
 ('AUBS', 223),
 ('AAPA', 218),
 ('ADTU', 199),
 ('JEBR', 192),
 ('ADPT', 155),
 ('PDBG', 154),
 ('AAPP', 149),
 ('AAPB', 116),
 ('AAPD', 116),
 ('AAPO', 114),
 ('AAPK', 110),
 ('AAPL', 110),
 ('CRBR', 108),
 ('ADTI', 108),
 ('AAPG', 105),
 ('JVBS', 104),
 ('JNBR', 96),
 ('JUBS', 96),
 ('PDGG', 94),
 ('BUBR', 92),
 ('AAPM', 85),
 ('AEBR', 76),
 ('JNBG', 73),
 ('JEBS', 72),
 ('AAPW', 70),
 ('AWBG', 66),
 ('GABR', 62)]

In [55]:
Counter( [ _[:5] for _ in categorized['file_name'].values ] ).most_common(n=30)

[('AUBS0', 221),
 ('AAPSE', 175),
 ('AAPSG', 152),
 ('JEBRS', 108),
 ('AAPOF', 90),
 ('AAPJE', 83),
 ('AAPDQ', 66),
 ('AAPAD', 62),
 ('ADTU1', 62),
 ('AAPBU', 54),
 ('AAPPH', 52),
 ('ADTU3', 51),
 ('AAPJN', 50),
 ('TSMB0', 50),
 ('AAPWR', 49),
 ('AAPPD', 45),
 ('AAPCR', 43),
 ('ADPTS', 41),
 ('DQBS5', 40),
 ('DQBS8', 40),
 ('ADTU5', 38),
 ('AAPKJ', 37),
 ('AAPTS', 37),
 ('ADBR3', 36),
 ('CRBG0', 36),
 ('DQBS6', 36),
 ('JNBR9', 36),
 ('LABS0', 36),
 ('AAPAE', 35),
 ('PDBG2', 33)]

In [56]:
uncategorized = files[ ~files['categorized'] ]

In [57]:
uncategorized.shape

(6689, 5)

In [58]:
Counter( [ _[:1] for _ in uncategorized['file_name'].values ] ).most_common()

[('A', 2604),
 ('S', 1943),
 ('N', 512),
 ('O', 387),
 ('P', 341),
 ('W', 332),
 ('M', 146),
 ('H', 117),
 ('B', 70),
 ('F', 61),
 ('E', 48),
 ('Y', 42),
 ('J', 25),
 ('Z', 25),
 ('C', 23),
 ('Q', 10),
 ('I', 1),
 ('L', 1),
 ('D', 1)]

In [59]:
Counter( [ _[:2] for _ in uncategorized['file_name'].values ] ).most_common( 30 )

[('A9', 2590),
 ('SE', 781),
 ('SG', 628),
 ('NA', 512),
 ('OF', 363),
 ('PH', 255),
 ('WR', 251),
 ('SD', 186),
 ('SC', 133),
 ('SI', 107),
 ('MG', 90),
 ('WU', 81),
 ('HF', 60),
 ('SB', 58),
 ('PJ', 53),
 ('MA', 52),
 ('ED', 47),
 ('HN', 41),
 ('BG', 28),
 ('FT', 28),
 ('SA', 26),
 ('CU', 23),
 ('OD', 23),
 ('FG', 20),
 ('PK', 18),
 ('YB', 18),
 ('ZQ', 17),
 ('HD', 16),
 ('JL', 16),
 ('YD', 14)]

In [60]:
Counter( [ _[:3] for _ in uncategorized['file_name'].values ] ).most_common( 30 )

[('A90', 2590),
 ('SEM', 777),
 ('SGM', 608),
 ('NAU', 433),
 ('OFB', 358),
 ('WRB', 213),
 ('PHB', 203),
 ('SDM', 186),
 ('SCM', 133),
 ('SIM', 107),
 ('WUB', 74),
 ('HFB', 60),
 ('SBM', 58),
 ('NAI', 58),
 ('PJB', 53),
 ('MGB', 51),
 ('EDB', 47),
 ('MAB', 47),
 ('PHG', 37),
 ('HNB', 37),
 ('BGB', 28),
 ('FTP', 28),
 ('SAM', 24),
 ('CUB', 22),
 ('FGB', 20),
 ('ODB', 20),
 ('MGU', 19),
 ('YBB', 18),
 ('ZQM', 17),
 ('HDB', 16)]

In [61]:
Counter( [ _[:4] for _ in uncategorized['file_name'].values ] ).most_common( 30 )

[('A900', 2590),
 ('SEMD', 777),
 ('SGMD', 608),
 ('NAUG', 303),
 ('SDMD', 186),
 ('WRBG', 146),
 ('SCMD', 133),
 ('NAUI', 130),
 ('PHBS', 107),
 ('SIMD', 107),
 ('OFBS', 81),
 ('OFBR', 80),
 ('OFBM', 73),
 ('PHBG', 72),
 ('SBMD', 58),
 ('NAIG', 58),
 ('OFBE', 54),
 ('OFBU', 46),
 ('WUBG', 46),
 ('PHGG', 37),
 ('HFBE', 36),
 ('BGBS', 28),
 ('EDBG', 27),
 ('HNBG', 24),
 ('MABR', 24),
 ('MGBR', 24),
 ('SAMD', 24),
 ('WRBE', 22),
 ('HFBR', 20),
 ('PJBS', 20)]

In [62]:
Counter( [ _[:5] for _ in uncategorized['file_name'].values ] ).most_common( 30 )

[('SEMDD', 777),
 ('SGMDD', 608),
 ('A900S', 420),
 ('A900A', 310),
 ('NAUG0', 280),
 ('A900J', 230),
 ('A900P', 204),
 ('SDMDD', 186),
 ('A900L', 175),
 ('A900M', 174),
 ('A900G', 152),
 ('SCMDD', 133),
 ('NAUIQ', 130),
 ('A900K', 120),
 ('A900O', 114),
 ('SIMDD', 107),
 ('A900H', 102),
 ('A900B', 96),
 ('A900W', 91),
 ('A900N', 80),
 ('A900C', 59),
 ('NAIG0', 58),
 ('A900Z', 54),
 ('OFBS7', 52),
 ('A900T', 45),
 ('WRBGB', 42),
 ('A900R', 41),
 ('A900E', 37),
 ('A900D', 35),
 ('HFBE0', 32)]

In [63]:
Counter( [ _[:6] for _ in uncategorized['file_name'].values ] ).most_common( 30 )

[('SEMDD0', 221),
 ('SGMDD3', 166),
 ('A900SE', 155),
 ('SEMDD2', 136),
 ('NAUG01', 136),
 ('NAUG02', 112),
 ('SGMDD0', 111),
 ('A900PD', 110),
 ('A900AW', 98),
 ('SCMDD0', 97),
 ('A900JE', 95),
 ('SEMDD8', 93),
 ('A900SG', 79),
 ('SEMDD1', 74),
 ('A900OF', 73),
 ('A900SX', 73),
 ('SGMDD8', 73),
 ('SGMDD4', 72),
 ('SEMDD6', 68),
 ('A900NA', 59),
 ('SGMDD1', 59),
 ('A900AD', 58),
 ('A900WR', 58),
 ('SDMDD6', 56),
 ('SEMDD3', 56),
 ('SGMDD5', 55),
 ('A900MG', 51),
 ('A900HD', 50),
 ('SDMDD2', 50),
 ('SEMDD5', 49)]

## Line count analysis

In [64]:
files['file_dir'].value_counts()

SORCLIB.TXT1                  2767
ASL900.SCHEDULE.CNTL          2590
LISTLIB.TXT                   2286
ASL900.PROCLIB                2100
ASL900.SORCLIB.CBL            1872
SRCLIB.CBL                    1872
CNTLLIB.FOUND.LIST.TXT         541
DCLSTLIB.TXT                   228
ASL900.PROD.LIB.CNTL.ESY       203
ASL900.PROD.LIB.CNTL.FOCUS     157
DCLGENS.FOUND.LIST.TXT         120
ASLP00.CTLLIB.TXT               34
MISSING.CPYBOOKS.TXT            26
MISSING.SORCLIB.TXT             24
ASL900.PROD.LIB.CNTL.SAS        10
Name: file_dir, dtype: int64

In [65]:
files['line_count'] = files['line_count'].astype( int )

In [66]:
files.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14830 entries, 8318 to 6030
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   line_count     14830 non-null  int64 
 1   file_dir       14830 non-null  object
 2   file_name      14830 non-null  object
 3   file_name_len  14830 non-null  int64 
 4   categorized    14830 non-null  bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 593.8+ KB


In [67]:
line_count_analysis = files.groupby( 'file_dir' )['line_count'].describe().round().astype( int )

In [68]:
line_count_analysis.sort_values( '50%', ascending=False )

,count,mean,std,min,25%,50%,75%,max
file_dir,,,,,,,,
DCLSTLIB.TXT,228,6780,3302,621,4720,6120,8158,17681
LISTLIB.TXT,2286,6827,9196,112,2532,4303,7542,126184
SORCLIB.TXT1,2767,1233,1634,25,410,776,1364,18708
ASL900.SORCLIB.CBL,1872,1115,1578,25,406,694,1181,18708
SRCLIB.CBL,1872,1115,1578,25,406,694,1181,18708
MISSING.SORCLIB.TXT,24,952,2580,18,168,258,492,12745
ASL900.PROD.LIB.CNTL.ESY,203,106,105,12,42,72,118,631
DCLGENS.FOUND.LIST.TXT,120,77,94,21,37,55,87,959
MISSING.CPYBOOKS.TXT,26,71,93,3,28,46,80,485


# Analysis/transformation of IDMS data dictionary file "IDMS_RECORDS_cpy.txt"

## Initial cleaning

In [69]:
# !head -50 IDMS_RECORDS_cpy.txt

# p = Path( "2022-10-14_Whitespace_stripped.txt" )

# raw_text = p.read_text(encoding='latin1')

# len( raw_text )

# print( raw_text[:1000] )

# 

# p = re.compile( r'^1REPORT.*?^0$', flags = re.MULTILINE | re.DOTALL )

# len( p.findall( raw_text ) )

# removed_headers = p.sub( "", raw_text )

# len( removed_headers )

In [70]:
out = Path( "2022-10-14_Stripped_IDMS_RECORDS.txt" )

In [71]:
#out.write_text( removed_headers )

In [72]:
# reload after some hand-editing
removed_headers = out.read_text()

In [73]:
len( removed_headers )

15348439

In [74]:
lines = removed_headers.split( '\n' )

In [75]:
len( lines )

171643

In [76]:
p = re.compile( r'^\s+' )

In [77]:
p.findall( lines[0] )

[' ']

In [78]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[171642]

In [79]:
lines = [ _ for _ in lines if _ != "" ]

In [80]:
len( lines )

171642

In [81]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[]

In [82]:
leading_space_count = [ len( p.findall( _ )[0] ) for _ in lines if len( p.findall( _ ) ) > 0 ]

In [83]:
Counter( leading_space_count )

Counter({1: 3661,
         5: 18569,
         12: 31763,
         14: 2902,
         9: 31042,
         26: 13934,
         11: 62916,
         13: 63,
         10: 6756,
         27: 3,
         22: 33})

## Create grouping variables based on position

In [84]:
table_index = -1
synonym_index = -1
table_indices = [0]*len( lines )
synonym_indices = [0]*len( lines )
modifier_target_lines = [0]*len( lines )
modified_line = -1
modifier_line_prefix = " "*26

for i in range( len( lines ) ):
    if lines[i][1] != ' ':
        table_index += 1
        synonym_index = -1
    if lines[i].startswith( '     RECORD'):
        synonym_index += 1
    if modified_line < 0 and lines[i].startswith( modifier_line_prefix ):
        # first time we're seeing a modifier?
        # Record the line that is being modified
        modified_line = i - 1
    if modified_line > 0 and not lines[i].startswith( modifier_line_prefix ):
        modified_line = -1

    modifier_target_lines[ i ] = modified_line
    table_indices[ i ] = table_index
    synonym_indices[ i ] = synonym_index

In [85]:
#table_indices

In [86]:
len( leading_space_count ) == len( lines )

True

In [87]:
import pandas as pd

In [88]:
lines_df = pd.DataFrame( dict(
    leading_space_count = leading_space_count,
    table_index = table_indices,
    synonym_index = synonym_indices,
    modifies_line = modifier_target_lines,
    line = lines ))

In [89]:
lines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171642 entries, 0 to 171641
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   leading_space_count  171642 non-null  int64 
 1   table_index          171642 non-null  int64 
 2   synonym_index        171642 non-null  int64 
 3   modifies_line        171642 non-null  int64 
 4   line                 171642 non-null  object
dtypes: int64(4), object(1)
memory usage: 6.5+ MB


In [90]:
lines_df[ lines_df[ 'modifies_line' ] > 0]

,leading_space_count,table_index,synonym_index,modifies_line,line
58,26,1,0,57,REDEFINES ...
60,26,1,0,59,OCCURS 4
70,26,1,0,69,VALUE ...
72,26,1,0,71,VALUE ...
122,26,2,0,121,REDEFINES ...
...,...,...,...,...,...
171557,26,3659,0,171556,REDEFINES ...
171569,26,3659,0,171568,OCCURS 5
171617,26,3660,0,171616,REDEFINES ...
171624,26,3660,0,171623,REDEFINES ...


In [91]:
lines_df[ 'modifies_line' ].value_counts().value_counts()

1         12728
2           245
5            22
4            14
6            13
9            12
10           10
3            10
7             9
8             8
11            6
13            2
157705        1
18            1
Name: modifies_line, dtype: int64

In [92]:
lines_df[ lines_df['leading_space_count'] == 1 ]

,leading_space_count,table_index,synonym_index,modifies_line,line
0,1,0,-1,-1,ABSDOMN VER 1 ...
11,1,1,-1,-1,ACCRLINK VER 1 ...
104,1,2,-1,-1,ACCRLINK VER 2 ...
168,1,3,-1,-1,ACCT VER 1 ...
184,1,4,-1,-1,ACCTG-CNTL VER 1 ...
...,...,...,...,...,...
171288,1,3656,-1,-1,502-REPT-DTL-RCRD VER 1 ...
171366,1,3657,-1,-1,502-REPT-DTL-RCRD VER 2 ...
171433,1,3658,-1,-1,502-REPT-DTL-RCRD VER 4 ...
171505,1,3659,-1,-1,502-TEMP-DTL-RCRD VER 1 ...


In [93]:
text_clumps = lines_df.groupby( 'leading_space_count' ).apply( lambda group: group['line'].to_list() )

In [94]:
text_clumps.info()

<class 'pandas.core.series.Series'>
Int64Index: 11 entries, 1 to 27
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
11 non-null     object
dtypes: object(1)
memory usage: 176.0+ bytes


In [95]:
[ len( _ ) for _ in text_clumps ]

[3661, 18569, 31042, 6756, 62916, 31763, 63, 2902, 33, 13934, 3]

In [96]:
lines_df['synonym_index'].value_counts()

 0    131085
-1     21060
 1     16657
 2      1988
 3       500
 4       248
 5        77
 6        27
Name: synonym_index, dtype: int64

## TABLE NAMES

In [97]:
text_clumps[1][:20]

[' ABSDOMN                          VER     1    32      D                                    10/15/87 09/29/87',
 ' ACCRLINK                         VER     1   176      D                                    03/09/98 02/21/84',
 ' ACCRLINK                         VER     2   176      D                                    05/15/98 05/15/98',
 ' ACCT                             VER     1    40      S                                    09/24/87 09/21/87',
 ' ACCTG-CNTL                       VER     1   100      D                                    02/07/89 05/26/87',
 ' ACH-RCRD                         VER     1   148      D                                             06/22/84',
 ' ACQD-PROP-MAP-RCRD               VER     1   141      C                                    07/31/87 06/30/87',
 ' ACQD-PROP-QUEUE-RCRD             VER     1   260      C                                    01/20/93 06/30/87',
 ' ACQD-PROP-SALES-RCRD             VER     1   286      D                              

In [98]:
Counter( [ len(_) for _ in text_clumps[1] ] )

Counter({109: 3661})

In [99]:
p = re.compile( r'\s(\S+)\s+VER\s+(\d+)' )

In [100]:
table_info_list = [ p.match( _ ).groups(1) for _ in text_clumps[1] ]

In [101]:
len( table_info_list )

3661

In [102]:
table_info_list[:10]

[('ABSDOMN', '1'),
 ('ACCRLINK', '1'),
 ('ACCRLINK', '2'),
 ('ACCT', '1'),
 ('ACCTG-CNTL', '1'),
 ('ACH-RCRD', '1'),
 ('ACQD-PROP-MAP-RCRD', '1'),
 ('ACQD-PROP-QUEUE-RCRD', '1'),
 ('ACQD-PROP-SALES-RCRD', '1'),
 ('ACQD-PROP-SOLD', '1')]

In [103]:
table_names, table_vers = zip(*table_info_list )

In [104]:
created_dates = pd.to_datetime( [ _[101:109] for _ in text_clumps[1] ] )

In [105]:
len( created_dates )

3661

In [106]:
updated_dates = pd.to_datetime( [ _[92:100] for _ in text_clumps[1] ], errors='coerce' )

In [107]:
updated_dates

DatetimeIndex(['1987-10-15', '1998-03-09', '1998-05-15', '1987-09-24',
               '1989-02-07',        'NaT', '1987-07-31', '1993-01-20',
               '1994-09-19', '1991-02-22',
               ...
               '2000-01-28', '1992-06-03', '1989-11-15', '1994-09-16',
               '1983-09-15', '2002-12-09',        'NaT', '2002-03-14',
               '2002-12-09', '2002-03-14'],
              dtype='datetime64[ns]', length=3661, freq=None)

In [108]:
table_info = pd.DataFrame( data=dict(
    table_names=table_names,
    table_vers=table_vers,
    created_dates=created_dates,
    updated_dates= updated_dates
))

In [109]:
table_info['created_dates'] = table_info['created_dates'].dt.date
table_info['updated_dates'] = table_info['updated_dates'].dt.date

In [110]:
table_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   table_names    3661 non-null   object
 1   table_vers     3661 non-null   object
 2   created_dates  3661 non-null   object
 3   updated_dates  3047 non-null   object
dtypes: object(4)
memory usage: 114.5+ KB


In [111]:
table_info.head(20)

,table_names,table_vers,created_dates,updated_dates
0,ABSDOMN,1,1987-09-29,1987-10-15
1,ACCRLINK,1,1984-02-21,1998-03-09
2,ACCRLINK,2,1998-05-15,1998-05-15
3,ACCT,1,1987-09-21,1987-09-24
4,ACCTG-CNTL,1,1987-05-26,1989-02-07
5,ACH-RCRD,1,1984-06-22,NaT
6,ACQD-PROP-MAP-RCRD,1,1987-06-30,1987-07-31
7,ACQD-PROP-QUEUE-RCRD,1,1987-06-30,1993-01-20
8,ACQD-PROP-SALES-RCRD,1,1994-09-01,1994-09-19
9,ACQD-PROP-SOLD,1,1989-04-26,1991-02-22


## Lines beginning with 5 spaces contain table metadata

In [112]:
text_clumps[5][:20]

['     PREPARED BY          AWMC11',
 '     REVISED BY           AWMC11',
 '     DESCRIPTION          USER DOMAIN',
 '     RECORD               ABSDOMN                                  VER     1 D  SUFFIX -4541',
 '     PREPARED BY          AA-KGM',
 '     REVISED BY           ASL034',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     1 D  PREFIX W019-',
 '     PREPARED BY          ASL70M',
 '     REVISED BY           ASL70M',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     SAME AS              ACCRLINK                                 VER     1',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     2 D  PREFIX W019-',
 '     PREPARED BY          ASL052']

In [113]:
p = re.compile( r'     (\S+)' )

In [114]:
Counter( [ p.match( _ ).groups(1)[0] for _ in text_clumps[5] ] )

Counter({'PREPARED': 3629,
         'REVISED': 3032,
         'DESCRIPTION': 3176,
         'RECORD': 4126,
         'RFA-ID': 729,
         'DEFINITION': 1166,
         'SAME': 573,
         'IN': 1737,
         'COMMENT': 203,
         'PURPOSE': 63,
         'PRIMARY': 27,
         'FMHA-TRANS': 19,
         'LANGUAGE': 44,
         'SUB-SYSTEM': 2,
         'MODE': 39,
         'EGEN': 1,
         'USER': 1,
         'MEDIUM': 1,
         'FREQUENCY': 1})

### Pull DESCRIPTION one-liners

In [115]:
p = re.compile( r'^ (\S+)\s+VER\s+(\d+).*?DESCRIPTION\s+(.*?)$', flags=re.DOTALL | re.MULTILINE ) 

In [116]:
desc_table_names, desc_table_vers, desc_descs = zip( *p.findall( removed_headers ) )

In [117]:
len( desc_table_names )

3184

In [118]:
len( desc_table_vers )

3184

In [119]:
len( desc_table_names) 

3184

In [120]:
descs = pd.DataFrame( dict(
    table_names = desc_table_names,
    table_vers = desc_table_vers,
    descriptions = desc_descs ))

In [121]:
table_info.shape

(3661, 4)

In [122]:
descs.shape

(3184, 3)

In [123]:
merged_table_info = table_info.merge( descs, on=['table_names', "table_vers"], how='left' )

In [124]:
merged_table_info

,table_names,table_vers,created_dates,updated_dates,descriptions
0,ABSDOMN,1,1987-09-29,1987-10-15,USER DOMAIN
1,ACCRLINK,1,1984-02-21,1998-03-09,ACCRUAL LINK RECORD
2,ACCRLINK,2,1998-05-15,1998-05-15,ACCRUAL LINK RECORD
3,ACCT,1,1987-09-21,1987-09-24,DFIS ACCOUNT
4,ACCTG-CNTL,1,1987-05-26,1989-02-07,ACCOUNTING MONTH ACTIVITY TOTALS
...,...,...,...,...,...
3656,502-REPT-DTL-RCRD,1,1984-06-28,2002-12-09,COPY MODULE FOR 502-REPORT
3657,502-REPT-DTL-RCRD,2,2003-06-13,NaT,COPY MODULE FOR 502-REPORT
3658,502-REPT-DTL-RCRD,4,2002-03-12,2002-03-14,COPY MODULE FOR 502-REPORT
3659,502-TEMP-DTL-RCRD,1,1989-01-30,2002-12-09,WORK RECORD FOR 502 REPORT


### Pull synonym table names, prefixes and suffixes

In [125]:
suffix_df = lines_df[ lines_df['leading_space_count'] == 5 ]

In [126]:
suffix_df.shape

(18569, 5)

In [127]:
suffix_df = suffix_df[ suffix_df['line'].str.startswith( '     RECORD' ) ]

In [128]:
suffix_df = suffix_df.copy()

In [129]:
suffix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4126 entries, 4 to 171583
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   leading_space_count  4126 non-null   int64 
 1   table_index          4126 non-null   int64 
 2   synonym_index        4126 non-null   int64 
 3   modifies_line        4126 non-null   int64 
 4   line                 4126 non-null   object
dtypes: int64(4), object(1)
memory usage: 193.4+ KB


In [130]:
name_start_pos = 26
name_end_pos = 67
affix_type_start_pos = 80
affix_type_end_pos = 86
affix_start_pos = 87

In [131]:
suffix_df['synonym_name'] = [ _[26:67].strip() for _ in suffix_df['line'].values ]

In [132]:
suffix_df['affix_type'] = [ _[80:86] if len(_) > 80 else "" for _ in suffix_df['line'].values  ]

In [133]:
suffix_df['affix'] = [ _[87:] if len(_) > 80 else "" for _ in suffix_df['line'].values  ]

#### There exists 4126 total synonyms

In [286]:
suffix_df.shape

(4126, 8)

In [134]:
suffix_df.head(40)

,leading_space_count,table_index,synonym_index,modifies_line,line,synonym_name,affix_type,affix
4,5,0,0,-1,RECORD ABSDOMN ...,ABSDOMN,SUFFIX,-4541
21,5,1,0,-1,RECORD ACCRLINK ...,ACCRLINK,PREFIX,W019-
115,5,2,0,-1,RECORD ACCRLINK ...,ACCRLINK,PREFIX,W019-
175,5,3,0,-1,RECORD ACCT ...,ACCT,SUFFIX,-821
194,5,4,0,-1,RECORD ACCTG-CNTL ...,ACCTG-CNTL,SUFFIX,-MTH
215,5,5,0,-1,RECORD ACH-RCRD ...,ACH-RCRD,PREFIX,W138-
233,5,6,0,-1,RECORD ACQD-PROP-MAP-RCRD ...,ACQD-PROP-MAP-RCRD,SUFFIX,-W01
271,5,7,0,-1,RECORD ACQD-PROP-QUEUE-RCRD...,ACQD-PROP-QUEUE-RCRD,SUFFIX,-M01
278,5,7,1,-1,RECORD SYNONYM ACQD-PROP-QUEUE-RCRD...,ACQD-PROP-QUEUE-RCRD-WK,SUFFIX,-WK
289,5,8,0,-1,RECORD ACQD-PROP-SALES-RCRD...,ACQD-PROP-SALES-RCRD,PREFIX,WK7-


In [140]:
# temp = list( suffix_df['line'] )

# temp[:5]

# temp[0][67:76]

# temp[0].index("VER")

# #p = re.compile( r' ' )

# #suffix_df.groupby( 'table_index' ).apply( , axis=1, result_type='expand' )

## Lines beginning with 9 spaces indicate "COPIED IN PROGRAM", "FILE", AND "SUBSCHEMA"

In [141]:
#text_clumps[9][-200:]
#text_clumps[9][-20:]

In [142]:
p = re.compile( r'\s+(\S+)' )

In [143]:
Counter( [ p.match( _ ).groups(1)[0] for _ in text_clumps[9] if p.match( _ ) ] )

Counter({'COPIED': 29058, 'SUBSCHEMA': 1834, 'FILE': 70, 'LANGUAGE': 80})

In [144]:
#sorted( [ _ for _ in text_clumps[9] if "FILE" in _ ] )

In [145]:
#[ _ for _ in text_clumps[9] if "LANGUAGE" in _ ]

In [146]:
#[ _ for _ in text_clumps[9] if "SUBSCHEMA" in _ ]

In [147]:
#[ _ for _ in text_clumps[9] if "COPIED" in _ ]

In [148]:
lines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171642 entries, 0 to 171641
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   leading_space_count  171642 non-null  int64 
 1   table_index          171642 non-null  int64 
 2   synonym_index        171642 non-null  int64 
 3   modifies_line        171642 non-null  int64 
 4   line                 171642 non-null  object
dtypes: int64(4), object(1)
memory usage: 6.5+ MB


In [149]:
selection_criteria = ( lines_df['leading_space_count'] == 9 ) & lines_df['line'].str.contains( "COPIED" )

In [150]:
program_table_df = lines_df.loc[ selection_criteria, 'line' ].str.extract( r'(\S+)\s+VER\s+(\d+)$' )

In [151]:
program_table_df.columns = ['copied_in_program', 'copied_in_program_ver']

In [152]:
program_table_df

,copied_in_program,copied_in_program_ver
22,SDMDD204,1
23,SFMDD100,1
24,SDMDD170,1
25,SDMDD361,1
26,SDMDD362,1
...,...,...
171444,SBDLD101,5
171513,SDDLD355,5
171514,SDDLD355,3
171515,SDDLD355,1


In [153]:
program_table_df = lines_df.merge( program_table_df, left_index=True, right_index=True, how='right' )

In [154]:
program_table_df

,leading_space_count,table_index,synonym_index,modifies_line,line,copied_in_program,copied_in_program_ver
22,9,1,0,-1,COPIED IN PROGRAM SDMDD204 ...,SDMDD204,1
23,9,1,0,-1,COPIED IN PROGRAM SFMDD100 ...,SFMDD100,1
24,9,1,0,-1,COPIED IN PROGRAM SDMDD170 ...,SDMDD170,1
25,9,1,0,-1,COPIED IN PROGRAM SDMDD361 ...,SDMDD361,1
26,9,1,0,-1,COPIED IN PROGRAM SDMDD362 ...,SDMDD362,1
...,...,...,...,...,...,...,...
171444,9,3658,0,-1,COPIED IN PROGRAM SBDLD101 ...,SBDLD101,5
171513,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,5
171514,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,3
171515,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,1


# Thirty most popular programs cited in "CALLED IN PROGRAM" statements

In [155]:
program_table_df['copied_in_program'].value_counts().head(30)

SBDLD041    249
SGDLD711    235
SBDLD101    233
SBDLD361    210
SDDLD131    207
SGDLD365    200
SDDLD207    194
SDDLD355    168
SGDLD631    160
SCDLD042    160
SDDLD201    156
SBDLD005    155
SCDLD041    151
SDDLD208    150
SGDLD731    148
SBDLD008    148
SDDLD530    146
SCDLD082    142
SCDLD121    141
SEDLD219    140
SEDLD217    140
SGDLD661    136
SGDLD741    135
SDDLD181    132
SFDLD112    130
SFDLD121    129
SBDLD504    128
SEDLD218    121
SJDLD001    118
SDDLD214    118
Name: copied_in_program, dtype: int64

In [156]:
def GetCode( prog_name ):
    for code in valid_codes:
        if prog_name.startswith( code ):
            return code
    return None

In [157]:
program_table_df['subsystem'] = program_table_df['copied_in_program'].apply( GetCode )

## Twenty most popular subsystems cited in "CALLED IN PROGRAM" statements

1. AU - UPDATE PROGRAMS
2. TS - Rural Community Facility Tracking System (RCFTS)
3. BA - UPDATES AND PRINTOUTS
4. AD - DISCREPANCY PROCESSING AND ACCOUNT INFORMATION
5. AW - ONLINE LOAN HISTORY
6. LQ - REPACK BORROWERS FILE [Food repack facilities?](https://naturalchoicefoods.com/private-label-frozen-food-repacker/)
7. AR - STATE/COUNTY FILE CHANGES
8. FF - TERMINAL APPLICATION
9. DB - DATABASE MAINTENANCE - IDMS-DC USERS PASSWORD
10. ER - ACCOUNT STATUS

In [158]:
subsystem_counts = program_table_df['subsystem'].value_counts().to_frame()

In [159]:
subsystem_counts = subsystem_counts.reset_index()
subsystem_counts.columns = ['code', 'subsystem_table_count' ]

In [160]:
table_subsystem_counts = subsystem_counts.merge( subsystems )

In [161]:
table_subsystem_counts

,code,subsystem_table_count,plas_subsystem
0,AU,2143,UPDATE PROGRAMS
1,TS,1672,RCFTS
2,BA,1512,UPDATES AND PRINTOUTS
3,AD,581,DISCREPANCY PROCESSING AND ACCOUNT INFORMATION...
4,AW,333,ONLINE LOAN HISTORY
5,LQ,317,REPACK BORROWERS FILE
6,AR,219,STATE/COUNTY FILE CHANGES
7,FF,169,TERMINAL APPLICATION
8,DB,54,DATABASE MAINTENANCE - IDMS-DC USERS PASSWORD ...
9,ER,51,ACCOUNT STATUS


In [162]:
program_table_df

,leading_space_count,table_index,synonym_index,modifies_line,line,copied_in_program,copied_in_program_ver,subsystem
22,9,1,0,-1,COPIED IN PROGRAM SDMDD204 ...,SDMDD204,1,None
23,9,1,0,-1,COPIED IN PROGRAM SFMDD100 ...,SFMDD100,1,None
24,9,1,0,-1,COPIED IN PROGRAM SDMDD170 ...,SDMDD170,1,None
25,9,1,0,-1,COPIED IN PROGRAM SDMDD361 ...,SDMDD361,1,None
26,9,1,0,-1,COPIED IN PROGRAM SDMDD362 ...,SDMDD362,1,None
...,...,...,...,...,...,...,...,...
171444,9,3658,0,-1,COPIED IN PROGRAM SBDLD101 ...,SBDLD101,5,None
171513,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,5,None
171514,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,3,None
171515,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,1,None


In [163]:
table_to_program_map = program_table_df.groupby( 'table_index' )['copied_in_program'].agg( lambda x: sorted( list( set( x ) ) ) )

In [164]:
table_to_program_map.name = 'program_list'

In [165]:
table_to_program_map

table_index
1       [SCMDD021, SDMDD106, SDMDD160, SDMDD161, SDMDD...
6                                        [KNDG01, KNDU02]
7                                                [KNDU02]
11                                             [SGDLD365]
14      [SBDLD161, SDDLD215, SFDLD111, SFDLD112, SFDLD...
                              ...                        
3647    [SDMDD106, SDMDD160, SDMDD161, SDMDD170, SDMDD...
3649    [SDMDD106, SDMDD160, SDMDD161, SDMDD170, SDMDD...
3656    [SBDLD101, SBDLD142, SBMDD010, SCMDD021, SDDLD...
3658                                 [SBDLD101, SBDLD142]
3659                                           [SDDLD355]
Name: program_list, Length: 1844, dtype: object

In [166]:
per_table_program_count = table_to_program_map.apply( len )

In [167]:
per_table_program_count.name = 'per_table_program_count'

In [168]:
table_to_program_map = pd.concat( ( table_to_program_map, per_table_program_count ), axis=1 )

# Twenty tables most CALLED IN PROGRAM

Big winner: table 220

In [169]:
table_to_program_map.sort_values( "per_table_program_count", ascending=False ).head( 20 )

,program_list,per_table_program_count
table_index,,
220,"[ACDG001, ACDG003, ACDG007, ACDG010, ACDG011, ...",1124
565,"[ADDUUD01, ADDUUD02, ADDUUD03, BADULK22, PGDU0...",302
568,"[ADDUUD01, ADDUUD02, ADDUUD03, BADULK22, SADLD...",289
2360,"[BOBDIAL, BOBDIAL2, BOBDIAL3, CHUCK1, PGDG02, ...",279
933,"[ADDGCALC, ADDI0BB1, IMDDUANE, IMDE1F01, IMDE1...",277
1191,"[ADDUUD01, ADDUUD02, ADDUUD03, BADULK22, SADLD...",275
934,"[ADDI0BB1, ADDI0CA1, IMDDUANE, IMDE1F01, IMDE1...",270
935,"[ADDGCALC, ADDI0BB1, ADDI0CA1, IMDDUANE, IMDE1...",269
2703,"[IMDDUANE, IMDE1G10, IMDE1S01, IMDE3B01, IMDE3...",251


In [170]:
table_to_subsystem_map = program_table_df.dropna( subset=['subsystem'] ).groupby( 'table_index' )['subsystem'].agg( lambda x: sorted( list( set(x) ) ) )

In [171]:
table_to_subsystem_map

table_index
6                       [KN]
7                       [KN]
23      [AD, AO, AR, AW, OA]
24                      [AD]
25          [AD, AO, AR, AW]
                ...         
3596                    [MU]
3597                    [MU]
3598                    [MU]
3644                    [AD]
3645                    [AD]
Name: subsystem, Length: 704, dtype: object

In [172]:
per_table_subsystem_count = table_to_subsystem_map.apply( len )

In [173]:
per_table_subsystem_count.name = "per_table_subsystem_count"

In [174]:
table_to_subsystem_map = pd.concat( ( table_to_subsystem_map, per_table_subsystem_count ), axis=1 )

# Twenty most cross-functional tables

Winners:
* 2067
* 220
* 743
* 771

In [175]:
table_to_subsystem_map.sort_values("per_table_subsystem_count", ascending=False).head(20)

,subsystem,per_table_subsystem_count
table_index,,
2067,"[AD, AE, AH, AI, AR, AU, BA, BD, DA, DB, DE, D...",26
220,"[AD, AO, AP, AR, AS, AU, AW, BA, BE, CH, DB, F...",22
743,"[AE, AI, AU, DA, DE, ER, GF, GL, JF, JN, JT, K...",16
771,"[AE, AI, AU, DA, DE, ER, GF, GL, JF, JN, JT, K...",16
733,"[AE, AI, AU, DA, DE, ER, JN, LA, LC, LQ]",10
522,"[AH, AI, AU, ER, GF, GL, JF, JT, KM]",9
709,"[AH, AR, AU, BA, DA, ER, LA, LQ]",8
734,"[AD, AI, AU, DA, DE, ER, LA, LC]",8
740,"[AI, AU, DA, DE, DH, ER, LA, LI]",8


# Any letter code patterns in the for the programs without subsystems?

Important patterns:

* IMD* - 5162
* S?DLD* - 14149
* BGD* - 753

In [176]:
no_subsystem_categorization = program_table_df.loc[ program_table_df['subsystem'].isna(), 'copied_in_program' ]

In [177]:
no_subsystem_categorization.str.slice( stop=1 ).value_counts().head(20)

S    15082
I     5162
B      898
A      268
C      100
H       87
P       11
V        8
M        6
D        4
Name: copied_in_program, dtype: int64

In [178]:
no_subsystem_categorization.str.slice( stop=2 ).value_counts().head(20)

IM    5162
SB    4394
SD    3952
SG    2602
SC    1014
SA     967
BG     753
SE     661
SJ     647
SF     620
AC     268
SH     137
CT     100
HN      87
SI      50
BL      45
BN      43
SL      36
BK      32
BO      25
Name: copied_in_program, dtype: int64

In [179]:
no_subsystem_categorization.str.slice( stop=3 ).value_counts().head(20)

IMD    5162
SBD    4337
SDD    3435
SGD    2446
SAD     960
SCD     949
BGD     753
SJD     647
SED     587
SFD     569
SDM     516
ACD     235
SGM     142
SHD     137
CTD     100
HND      81
SEM      74
SCM      65
SBM      57
SID      50
Name: copied_in_program, dtype: int64

In [180]:
no_subsystem_categorization.str.slice( stop=4 ).value_counts().head(20)

SBDL    4337
SDDL    3435
SGDL    2446
IMDU    2325
IMDG    1430
IMDE    1269
SADL     960
SCDL     949
SJDL     647
SEDL     587
SFDL     569
SDMD     516
BGDI     428
BGDU     272
SGMD     142
ACDG     142
SHDL     137
ACDU      93
HNDI      81
SEMD      74
Name: copied_in_program, dtype: int64

In [181]:
no_subsystem_categorization.str.slice( stop=5 ).value_counts().head(20)

SBDLD    4337
SDDLD    3435
SGDLD    2442
SADLD     960
SCDLD     949
IMDU1     711
SJDLD     647
SEDLD     587
SFDLD     569
SDMDD     516
IMDU3     470
IMDE3     357
IMDER     295
IMDG1     285
BGDI1     266
BGDU0     248
IMDG3     219
IMDU5     217
IMDG5     191
IMDU4     187
Name: copied_in_program, dtype: int64

In [182]:
# nothing here
#no_subsystem_categorization.str.slice( stop=6 ).value_counts().head(20)

In [183]:
no_subsystem_categorization.str.contains( 'DLD' ).sum()

14149

# Pivot out subsystem annotations

In [184]:
program_table_df

,leading_space_count,table_index,synonym_index,modifies_line,line,copied_in_program,copied_in_program_ver,subsystem
22,9,1,0,-1,COPIED IN PROGRAM SDMDD204 ...,SDMDD204,1,None
23,9,1,0,-1,COPIED IN PROGRAM SFMDD100 ...,SFMDD100,1,None
24,9,1,0,-1,COPIED IN PROGRAM SDMDD170 ...,SDMDD170,1,None
25,9,1,0,-1,COPIED IN PROGRAM SDMDD361 ...,SDMDD361,1,None
26,9,1,0,-1,COPIED IN PROGRAM SDMDD362 ...,SDMDD362,1,None
...,...,...,...,...,...,...,...,...
171444,9,3658,0,-1,COPIED IN PROGRAM SBDLD101 ...,SBDLD101,5,None
171513,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,5,None
171514,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,3,None
171515,9,3659,0,-1,COPIED IN PROGRAM SDDLD355 ...,SDDLD355,1,None


# Map tables to PLAS subsystems

In [185]:
table_susbystem_pivot = program_table_df.pivot_table(
    index='table_index',
    columns='subsystem',
    values='leading_space_count', #something guaranteed not to be empty
    aggfunc='count'
).fillna(0).astype(bool).astype(int)

In [186]:
table_susbystem_pivot.columns

Index(['AD', 'AE', 'AH', 'AI', 'AO', 'AP', 'AR', 'AS', 'AU', 'AW', 'BA', 'BD',
       'BE', 'CH', 'DA', 'DB', 'DE', 'DF', 'DH', 'ER', 'FA', 'FF', 'GF', 'GL',
       'JF', 'JN', 'JT', 'KI', 'KM', 'KN', 'LA', 'LC', 'LI', 'LQ', 'MU', 'OA',
       'PA', 'PE', 'RC', 'RI', 'TA', 'TE', 'TS', 'VA'],
      dtype='object', name='subsystem')

In [187]:
len(table_susbystem_pivot.columns )

44

In [188]:
table_susbystem_pivot = table_susbystem_pivot[ table_subsystem_counts['code'].values ]

# 10's through 12's are data elements

In [189]:
text_clumps[10][-10:]

['          14500   05 DTE-LST-CNCLTN-RDA                       9(6)                             DISPLAY',
 '          14600   05 PRIN-INSTLMT-BOND-ACCTS-RDA              S9(7)V99                         COMP-3',
 '          14700   05 INSTLMT-OC-DATA-320-RDA                                                   DISPLAY',
 '          14800    10 DTE-INSTLMT-DUE-320-RDA                 9(6)                             DISPLAY',
 '          14900    10 INSTLMT-DUE-AMT-320-RDA                 S9(06)V99                        COMP-3',
 '          15000    10 INSTLMT-TYP-CDE-320-RDA                 9(1)                             DISPLAY',
 '          15100    10 MTURD-CDE-DIR-RE-320-RDA                9(1)                             DISPLAY',
 '          15200   05 RCVB-AMT-DUE-RDA                         S9(09)V99                        COMP-3',
 '          15300   05 RCVB-AMT-PD-RDA                          S9(09)V99                        COMP-3',
 '          15400   05 FILLER            

In [190]:
text_clumps[11][-10:]

['           4600   05 W502-RGSTR-TYP-LST-CHNG                  9(2)                             DISPLAY',
 '           4700   05 FILLER                                   X(0014)                          DISPLAY',
 '           4800   05 W502-DTE-APPRVL-OBLGN                    9(5)                             DISPLAY',
 '           4900   05 W502-ETHNCTY-TYP-CDE                     X                                DISPLAY',
 '           5000   05 W502-RACE-TYP-CDE                        X                                DISPLAY',
 '           5100   05 W502-RACE-DTRMNTN-CDE                    X                                DISPLAY',
 '           5200   05 W502-ST-CDE                              9(2)                             DISPLAY',
 '           5300   05 FILLER                                   X(0002)                          DISPLAY',
 '           5400   05 W502-CR-REPT-FEE-CDE                     9(1)                             DISPLAY',
 '           5500   05 W502-PROJ-NBR-

In [191]:
text_clumps[12][-10:]

['            900    10 W502-PPB-CDE-1ST                        9                                DISPLAY',
 '            100   05 W502-RCRD-TYP                            9(2)                             DISPLAY',
 '            200   05 W502-RGSTR-TYP-CDE                       9(2)                             DISPLAY',
 '            300   05 W502-MO-OBLGN                            9(2)                             DISPLAY',
 '            400   05 W502-FY                                  9(2)                             DISPLAY',
 '            500   05 W502-APROPTN-MAJ-CLS-CDE                                                  DISPLAY',
 '            600    10 W502-APROPTN-CDE                        9(2)                             DISPLAY',
 '            700    10 W502-MAJ-CLS                            9(2)                             DISPLAY',
 '            800   05 W502-PROG-PLANG-BDGTG-CDE                                                 DISPLAY',
 '            900    10 W502-PPB-CDE-

In [192]:
all_data_elements = [ *text_clumps[10], *text_clumps[11], *text_clumps[12] ]

In [193]:
len( all_data_elements )

101435

## Get a count of all elements in the base (non-synonym version) of a table

* FILTER OUT SYNONYMS HERE

In [194]:
all_elements = lines_df[ (lines_df['leading_space_count'] >= 10) & (lines_df['leading_space_count'] <= 12) & (lines_df['synonym_index'] == 0) ] 

In [195]:
lines_df[ 'synonym_index' ].value_counts()

 0    131085
-1     21060
 1     16657
 2      1988
 3       500
 4       248
 5        77
 6        27
Name: synonym_index, dtype: int64

In [196]:
all_elements.shape

(89288, 5)

In [197]:
all_elements.groupby( 'table_index' )['line'].count()

table_index
0        6
1       48
2       48
3        7
4       15
        ..
3656    57
3657    57
3658    57
3659    55
3660    55
Name: line, Length: 3601, dtype: int64

In [287]:
import json

In [289]:
css =  'transform: matrix(0.681373, 0, 0, 0.681373, 0, 0); transform-origin: -20.835px -712.288px; left: 20.835px; top: 712.288px; width: 569.998px; height: 37.716px;' 

In [300]:
dict( _.strip().split( ': ' ) for _ in css.split( ';' ) if _ )

{'transform': 'matrix(0.681373, 0, 0, 0.681373, 0, 0)',
 'transform-origin': '-20.835px -712.288px',
 'left': '20.835px',
 'top': '712.288px',
 'width': '569.998px',
 'height': '37.716px'}

In [198]:
all_elements[ 'synonym_index' ].value_counts()

0    89288
Name: synonym_index, dtype: int64

## Scrape data elements from lines with 13 spaces

In [199]:
len(text_clumps[13])

63

In [200]:
text_clumps[13][:20]

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY',
 '             1000 INT-RATE-CTGRY IS

In [201]:
data_elements_13 = [ _ for _ in text_clumps[13] if _[16] == ' ' ]

In [202]:
len( data_elements_13 )

9

In [203]:
data_elements_13

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY']

In [204]:
all_data_elements.extend( data_elements_13 )

In [205]:
len( all_data_elements )

101444

### Locations

* [0:15] - declaration step
* [15:62] - (\s+)(\d+) (\s+)
* [62:95] - data format
* [95:] - end

In [206]:
all_elements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89288 entries, 5 to 171641
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   leading_space_count  89288 non-null  int64 
 1   table_index          89288 non-null  int64 
 2   synonym_index        89288 non-null  int64 
 3   modifies_line        89288 non-null  int64 
 4   line                 89288 non-null  object
dtypes: int64(4), object(1)
memory usage: 4.1+ MB


In [207]:
p = re.compile( r'^\s+(\d+)(\s+)(\d+) (\S+)' )#\s+(\S+)?\s+(\S)$' )
#p = re.compile( r'^\s+(\d+)\s+(\d+) (\S+)\s+(\S+)?\s+(\S)$' )

In [208]:
p.match( all_data_elements[0] ).groups()

('10000', '   ', '05', 'AW-U-YR')

In [209]:
declaration_steps, indent, indent_number, field_name  = zip( *all_elements['line'].transform( lambda l: p.match( l ).groups() ).values )

In [210]:
data_types = all_elements['line'].transform( lambda l: l[62:95].strip() )

In [211]:
# not sure to what to call the "DISPLAY" etc that brings up the rear of these data field lines
end_types = all_elements['line'].transform( lambda l: l[95:].strip() )

In [212]:
len( data_types )

89288

In [213]:
len( indent )

89288

In [214]:
indent = [ len(_) for _ in indent ]

In [215]:
data_fields = pd.DataFrame(
    data = dict(
        declaration_step = declaration_steps,
        indent_space_count = indent,
        indent_number = indent_number,
        field_name = field_name,
        data_type = data_types,
        end = end_types
    ),
    index = all_elements.index
)

In [216]:
data_fields.sample( 10 )

,declaration_step,indent_space_count,indent_number,field_name,data_type,end
161719,400,3,05,LN-PRBLM-STAT-OLD-5A02,X(1),DISPLAY
100717,37400,3,05,FILLER,X(0032),DISPLAY
66758,1500,4,10,DTE-VIEWD-YR-4320,9(2),DISPLAY
143223,700,3,05,RCRD-TYP-CDE,X,DISPLAY
8091,1000,3,05,CARD-TYP,X(01),DISPLAY
73196,4000,3,05,D049-REAS-CDE-MFH,99,DISPLAY
12606,900,3,05,NME-BORR-LST-AFMS,X(20),DISPLAY
19821,200,3,05,D255-CTY-CDE-FMHA,9(3),DISPLAY
22079,1800,3,05,X013-TYP-OF-ASSTNC-CDE-MFH,999,DISPLAY
19317,1800,3,05,D013-VET-CDE,9(1),DISPLAY


In [217]:
data_fields[ ['indent_space_count', 'indent_number' ] ].value_counts()

indent_space_count  indent_number
3                   05               53625
4                   10               24046
5                   15                6156
7                   88                2718
3                   03                1375
6                   20                 906
3                   04                 151
                    02                 146
7                   25                 112
4                   07                  49
                    09                   4
dtype: int64

In [218]:
data_fields[ data_fields[ 'indent_number' ] =='09']

,declaration_step,indent_space_count,indent_number,field_name,data_type,end
111534,6100,4,09,DCSTR4,X(4),DISPLAY
111535,6200,4,09,DCSTR5,X(4),DISPLAY
112045,7100,4,09,DCSTR4,X(4),DISPLAY
112046,7200,4,09,DCSTR5,X(4),DISPLAY


In [219]:
unique_data_types = set( data_types )

In [220]:
unique_data_types = sorted( list( unique_data_types ), key=lambda t: ( len(t), t ) )

In [221]:
Counter( end_types )

Counter({'DISPLAY': 70534,
         'COMP-3': 14456,
         'COND': 2718,
         'COMP': 1454,
         'COMP         SYNC': 86,
         'BIT': 10,
         'POINTER': 13,
         'COMP-2': 15,
         'COMP-1': 2})

In [222]:
#Path( '2022-10-17_IDMS_COBOL_datatypes.txt' ).write_text( '\n'.join( unique_data_types ) )

In [223]:
temp = all_elements[ ['table_index', 'synonym_index' ] ]

In [224]:
temp

,table_index,synonym_index
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
...,...,...
171637,3660,0
171638,3660,0
171639,3660,0
171640,3660,0


In [225]:
data_fields = pd.concat( (temp, data_fields), axis=1 )

In [226]:
data_fields

,table_index,synonym_index,declaration_step,indent_space_count,indent_number,field_name,data_type,end
5,0,0,100,3,05,ST-CTY-CDE-FMHA-4541,,DISPLAY
6,0,0,200,4,10,ST-CDE-FMHA-4541,9(2),DISPLAY
7,0,0,300,4,10,CTY-CDE-FMHA-4541,9(3),DISPLAY
8,0,0,400,3,05,NME-CTY-4541,X(20),DISPLAY
9,0,0,500,3,05,LST-UPDTE-DTE-4541,9(06),DISPLAY
...,...,...,...,...,...,...,...,...
171637,3660,0,5100,3,05,W502-RACE-DTRMNTN-CDE,X,DISPLAY
171638,3660,0,5200,3,05,W502-ST-CDE,9(2),DISPLAY
171639,3660,0,5300,3,05,FILLER,X(0002),DISPLAY
171640,3660,0,5400,3,05,W502-CR-REPT-FEE-CDE,9(1),DISPLAY


## Scrape COMMENTS from lines beginning with 14 spaces (DEFINITION, COMMENT, and PURPOSE)

In [227]:
text_clumps[14][:20]

['              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THE AMOUNT IN THAT DFIS ACCOUNT.',
 '              100 AT THE END OF EACH ACCOUNTING MONTH BEFORE THE',
 '              200 STATE-CNTL AND THE CYCLE-CNTL RECORDS ARE RESET',
 '              300 TO ZERO FOR THE NEXT MONTH, THE ACTIVITY CONTROL',
 '              400 TOTALS FOR THE COMPLETED MONTH WILL BE MOVED',
 '              500 INTO THIS 15 MONTH ACCTG-CNTL FILE.',
 '              100 ACQUIRED PROPERTY QUEUE RECORD',
 '              100 STANDARD RECORD DESCRIPTION FOR ACQUIRED',
 '              200 PROPERTY TAPE FILE',
 '              100 PICTURE AND USAGE FOR SOME ELEMENTS HAVE BEEN OVERRIDDEN',
 '              200 TO CREATE RECORD AS ESTABLISHED DURING CONVERSION',
 '

In [228]:
comment_lines = lines_df[ lines_df['leading_space_count'] == 14 ]

In [229]:
comments = comment_lines.groupby( 'table_index' ).apply( lambda group: " ".join( _[18:] for _ in group['line'].values  ) )

In [230]:
comments[2]

'THIS RECORD IS USED TO PASS TO AND FROM INFORMATION REQUIRED TO COMPLETE AN ACCRUAL'

In [231]:
comments

table_index
1       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
2       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
3                        THE AMOUNT IN THAT DFIS ACCOUNT.
4       AT THE END OF EACH ACCOUNTING MONTH BEFORE THE...
7                          ACQUIRED PROPERTY QUEUE RECORD
                              ...                        
3656    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3657    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3658    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3659    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
3660    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
Length: 1307, dtype: object

## Merge table subsystem mapping

In [232]:
comments.name = 'comments'

In [233]:
merged_table_info = merged_table_info.merge( comments, how='left', left_index=True, right_index=True )

In [234]:
merged_table_info = merged_table_info.sort_values( ['created_dates', 'table_names', 'table_vers'] )

In [235]:
import numpy as np

In [236]:
table_susbystem_pivot[ table_susbystem_pivot == 0] = np.nan

In [237]:
new_col_names = [ two_letter_code_to_subsystem_desc_map[_] for _ in table_susbystem_pivot.columns ]

In [238]:
new_col_names[1] = 'RURAL COMMUNITY FACILITY TRACKING SYSTEM'

In [239]:
table_susbystem_pivot.columns = new_col_names

In [240]:
merged_table_info = merged_table_info.merge( table_susbystem_pivot, left_index=True, right_index=True, how='left' )

In [241]:
merged_table_info.to_excel( '2022-12-15_IDMS_table_descriptions.xlsx' )

In [242]:
suffix_df

,leading_space_count,table_index,synonym_index,modifies_line,line,synonym_name,affix_type,affix
4,5,0,0,-1,RECORD ABSDOMN ...,ABSDOMN,SUFFIX,-4541
21,5,1,0,-1,RECORD ACCRLINK ...,ACCRLINK,PREFIX,W019-
115,5,2,0,-1,RECORD ACCRLINK ...,ACCRLINK,PREFIX,W019-
175,5,3,0,-1,RECORD ACCT ...,ACCT,SUFFIX,-821
194,5,4,0,-1,RECORD ACCTG-CNTL ...,ACCTG-CNTL,SUFFIX,-MTH
...,...,...,...,...,...,...,...,...
171295,5,3656,0,-1,RECORD 502-REPT-DTL-RCRD ...,502-REPT-DTL-RCRD,PREFIX,W139-
171372,5,3657,0,-1,RECORD 502-REPT-DTL-RCRD ...,502-REPT-DTL-RCRD,PREFIX,W139-
171441,5,3658,0,-1,RECORD 502-REPT-DTL-RCRD ...,502-REPT-DTL-RCRD,PREFIX,W139-
171512,5,3659,0,-1,RECORD 502-TEMP-DTL-RCRD ...,502-TEMP-DTL-RCRD,PREFIX,W502-


In [243]:
all_table_info = merged_table_info.merge( suffix_df, left_index=True, right_on='table_index' )

In [244]:
all_table_info.head()

,table_names,table_vers,created_dates,updated_dates,descriptions,comments,UPDATE PROGRAMS,RURAL COMMUNITY FACILITY TRACKING SYSTEM,UPDATES AND PRINTOUTS,DISCREPANCY PROCESSING AND ACCOUNT INFORMATION INQUIRY,...,LIBRARY MAINTENANCE,ACQUIRED PROPERTY/LEASE ACCOUNTS,leading_space_count,table_index,synonym_index,modifies_line,line,synonym_name,affix_type,affix
27609,DB-STATISTICS,1,1981-08-25,2010-07-16,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,5,522,0,-1,RECORD DB-STATISTICS ...,DB-STATISTICS,,
27675,DB_STATISTICS,1,1981-08-25,2010-07-16,WORK RECORD FOR DBMG06 MAP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5,525,0,-1,RECORD DB_STATISTICS ...,DB_STATISTICS,,
27791,DC-AID-CONDITION-NAMES,1,1981-08-25,2010-07-16,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,5,532,0,-1,RECORD DC-AID-CONDITION-NAM...,DC-AID-CONDITION-NAMES,,
64343,MRB-,1,1981-08-25,2010-07-16,NaN,"THIS RECORD IS REQUIRED FOR MAPPING FUNCTIONS,...",NaN,NaN,NaN,NaN,...,NaN,NaN,5,1118,0,-1,RECORD MRB- ...,MRB-,,
109556,SSCTRL,1,1981-08-25,1994-10-22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5,2020,0,-1,RECORD SSCTRL ...,SSCTRL,,


In [245]:
all_table_info['synonym_index'].value_counts()

0    3661
1     375
2      66
3      15
4       6
5       2
6       1
Name: synonym_index, dtype: int64

In [246]:
data_fields['synonym_index'].value_counts()

0    89288
Name: synonym_index, dtype: int64

In [247]:
all_table_info = all_table_info.drop( columns=['leading_space_count', 'modifies_line', 'line' ] ).sort_index()

In [248]:
# all_table_info = all_table_info[ ['table_names', 'synonym_name', 'table_vers', 'created_dates', 'updated_dates',
#        'descriptions', 'comments',  'affix_type', 'affix'] ]

In [249]:
all_table_info.sample(10)

,table_names,table_vers,created_dates,updated_dates,descriptions,comments,UPDATE PROGRAMS,RURAL COMMUNITY FACILITY TRACKING SYSTEM,UPDATES AND PRINTOUTS,DISCREPANCY PROCESSING AND ACCOUNT INFORMATION INQUIRY,...,INVESTOR ACCOUNTS,FINANCE OFFICE REPORTING,PROCUREMENT PROCESS,LIBRARY MAINTENANCE,ACQUIRED PROPERTY/LEASE ACCOUNTS,table_index,synonym_index,synonym_name,affix_type,affix
27747,DBSTATS,1,1994-10-22,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,530,0,DBSTATS,,
13148,ALOTMT,2,1986-12-19,NaT,ALLOTMENT RECORD,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,254,0,ALOTMT,PREFIX,D003-
158453,WK-3R-SAVE-RCRD,1,1995-03-21,1995-04-27,WORK/MAP SAVE RECORD FOR 3R DIALOGS.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3340,0,WK-3R-SAVE-RCRD,SUFFIX,-3R-SV
164804,WK-555-XPNCESTAT,1,1986-09-12,1987-09-24,WORK RCD SCR CODE 555 EXP STAT SUBSYST,STATE OFFICE POSITIONS FIFTH SCREEN FOR SCREEN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3526,1,WK-555-XPNSTAT,SUFFIX,-M
21204,CHARGE,1,1987-05-12,1991-03-28,ONLINE HISTORY CHARGE RECORD.,AN ONLINE HISTORY RECORD REPRESENTING A CHARGE...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,406,0,CHARGE,SUFFIX,-OLH
38531,FARMS-CRBUR,1,1994-12-14,2000-08-08,CREDIT BUREAU,CONTAINS DATA FOR CREDIT BUREAU REPORTING,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,694,1,CRBUR,SUFFIX,-302
137320,WK-LI02-INQ,1,1993-10-29,1993-11-03,CERTFD LNDR WORK RCRD# 2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2728,0,WK-LI02-INQ,SUFFIX,-INQ2
91031,SDDLD355-CSTPROJ-WK,1,1994-01-21,1994-01-21,PREAPPLICATION COST WORK RECORD,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1702,0,SDDLD355-CSTPROJ-WK,PREFIX,W055-
1110,ADM-ADJMT-TRNSCTN-RCRD,1,1984-04-23,1988-02-18,ADMINISTRATION ADJUSTMENT TRANSACTION,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,0,ADM-ADJMT-TRNSCTN-RCRD,PREFIX,X034-
34548,FARMS-ACQD-PROP,9,1993-01-06,1993-01-08,ACQUIRED PROPERTY DATABASE RECORD,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,657,1,ACQD-PROP,,


In [250]:
data_fields

,table_index,synonym_index,declaration_step,indent_space_count,indent_number,field_name,data_type,end
5,0,0,100,3,05,ST-CTY-CDE-FMHA-4541,,DISPLAY
6,0,0,200,4,10,ST-CDE-FMHA-4541,9(2),DISPLAY
7,0,0,300,4,10,CTY-CDE-FMHA-4541,9(3),DISPLAY
8,0,0,400,3,05,NME-CTY-4541,X(20),DISPLAY
9,0,0,500,3,05,LST-UPDTE-DTE-4541,9(06),DISPLAY
...,...,...,...,...,...,...,...,...
171637,3660,0,5100,3,05,W502-RACE-DTRMNTN-CDE,X,DISPLAY
171638,3660,0,5200,3,05,W502-ST-CDE,9(2),DISPLAY
171639,3660,0,5300,3,05,FILLER,X(0002),DISPLAY
171640,3660,0,5400,3,05,W502-CR-REPT-FEE-CDE,9(1),DISPLAY


In [251]:
data_fields = data_fields.reset_index().rename( columns={'index':'line' } )

In [252]:
all_fields_info = data_fields.merge( all_table_info, how='left', on=['table_index', 'synonym_index' ] ).drop( columns=['descriptions', 'comments' ])

In [253]:
all_fields_info['field_name_stripped'] = all_fields_info.apply( lambda row: row['field_name'].replace( row['affix'], "" ), axis=1 )

In [254]:
len( all_fields_info )

89288

In [255]:
all_fields_info.columns

Index(['line', 'table_index', 'synonym_index', 'declaration_step',
       'indent_space_count', 'indent_number', 'field_name', 'data_type', 'end',
       'table_names', 'table_vers', 'created_dates', 'updated_dates',
       'UPDATE PROGRAMS', 'RURAL COMMUNITY FACILITY TRACKING SYSTEM',
       'UPDATES AND PRINTOUTS',
       'DISCREPANCY PROCESSING AND ACCOUNT INFORMATION INQUIRY',
       'ONLINE LOAN HISTORY', 'REPACK BORROWERS FILE',
       'STATE/COUNTY FILE CHANGES', 'TERMINAL APPLICATION',
       'DATABASE MAINTENANCE - IDMS-DC USERS PASSWORD UPDATE',
       'ACCOUNT STATUS', 'BORROWER ACCOUNTS',
       'SALE OF COMMUNITY FACILITY LOANS', 'ACCOUNT MATURING',
       'DEPOSIT FUND SYSTEM', 'OBLIGATIONS-TO-ALLOTMENT SUMMARY BALANCING',
       'ADMINISTRATIVE OFFSET', 'DAILY FILE UTILITY MAINTENANCE',
       'PROCESSED AND DISCREPANCY REGISTERS', 'BLOCKLIST',
       'DEFAULT PROCESSING', 'WEEKLY MERGES', 'PRELIMINARY REGISTER ACTIVITY',
       'REPORT CODE 732 - PROJECTED LIABILITY FOR

In [256]:
all_fields_info['affix'].value_counts().head(20)

         24890
D014-     1233
D270-     1071
D320-      546
-EX        419
W238-      357
-WK        353
-FR        342
D003-      339
D049-      313
X005-      297
-101       294
W005-      286
D227-      285
D023-      279
D036-      279
D015-      278
W013-      267
W174-      265
WK01-      263
Name: affix, dtype: int64

In [257]:
wanted_columns = ['line', 'table_names', 'table_index', 'table_vers', 'created_dates', 'updated_dates',
       'field_name_stripped', 'data_type', 'indent_number', 'indent_space_count',
       'declaration_step', 'end', 'field_name', ]

In [258]:
all_fields_info = all_fields_info[wanted_columns].copy()

In [259]:
all_fields_info.columns

Index(['line', 'table_names', 'table_index', 'table_vers', 'created_dates',
       'updated_dates', 'field_name_stripped', 'data_type', 'indent_number',
       'indent_space_count', 'declaration_step', 'end', 'field_name'],
      dtype='object')

In [260]:
# RENAME
all_fields_info.columns = [ 'line', 'table_name', 'table_index', 'table_vers', 'table_created_date', 'table_updated_date',
        'field_name',  'data_type', 'indent_number',
       'indent_space_count', 'declaration_step', 'end', 'raw_field_name',]

In [261]:
all_fields_info

,line,table_name,table_index,table_vers,table_created_date,table_updated_date,field_name,data_type,indent_number,indent_space_count,declaration_step,end,raw_field_name
0,5,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CTY-CDE-FMHA,,05,3,100,DISPLAY,ST-CTY-CDE-FMHA-4541
1,6,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CDE-FMHA,9(2),10,4,200,DISPLAY,ST-CDE-FMHA-4541
2,7,ABSDOMN,0,1,1987-09-29,1987-10-15,CTY-CDE-FMHA,9(3),10,4,300,DISPLAY,CTY-CDE-FMHA-4541
3,8,ABSDOMN,0,1,1987-09-29,1987-10-15,NME-CTY,X(20),05,3,400,DISPLAY,NME-CTY-4541
4,9,ABSDOMN,0,1,1987-09-29,1987-10-15,LST-UPDTE-DTE,9(06),05,3,500,DISPLAY,LST-UPDTE-DTE-4541
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89283,171637,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,RACE-DTRMNTN-CDE,X,05,3,5100,DISPLAY,W502-RACE-DTRMNTN-CDE
89284,171638,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,ST-CDE,9(2),05,3,5200,DISPLAY,W502-ST-CDE
89285,171639,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,FILLER,X(0002),05,3,5300,DISPLAY,FILLER
89286,171640,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,CR-REPT-FEE-CDE,9(1),05,3,5400,DISPLAY,W502-CR-REPT-FEE-CDE


# 22's 

In [262]:
text_clumps[22][:20]

['                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9']

# 26's: REDEFINES, OCCURS, VALUE

In [263]:
len( text_clumps[26] )

13934

In [264]:
all_elements[ 'leading_space_count' ]

5         12
6         12
7         12
8         12
9         12
          ..
171637    11
171638    11
171639    11
171640    11
171641    11
Name: leading_space_count, Length: 89288, dtype: int64

In [265]:
modifiers_lines_df = lines_df[ lines_df[ 'leading_space_count' ] == 26 ].copy()

In [266]:
modifiers_lines_df

,leading_space_count,table_index,synonym_index,modifies_line,line
58,26,1,0,57,REDEFINES ...
60,26,1,0,59,OCCURS 4
70,26,1,0,69,VALUE ...
72,26,1,0,71,VALUE ...
122,26,2,0,121,REDEFINES ...
...,...,...,...,...,...
171557,26,3659,0,171556,REDEFINES ...
171569,26,3659,0,171568,OCCURS 5
171617,26,3660,0,171616,REDEFINES ...
171624,26,3660,0,171623,REDEFINES ...


In [267]:
modifiers_lines_df['modifies_line'].value_counts().value_counts()

1     12730
2       244
5        22
4        14
6        13
9        12
10       10
7         9
3         9
8         8
11        6
13        2
18        1
Name: modifies_line, dtype: int64

In [268]:
modifiers_lines_df.loc[58,'line']

'                          REDEFINES               W019-RQST-AREA-MFH'

In [269]:
wanted_cols = ['modifier_type', 'modifier_value' ] 

In [270]:
modifiers_lines_df[ wanted_cols ] = modifiers_lines_df['line'].str.extract( r'^\s*(\S+ ?\S*)\s+(\S+.*)$')

In [271]:
modifiers_lines_df['modifier_type'] = modifiers_lines_df['modifier_type'].str.strip()

In [272]:
modifiers_lines_df = modifiers_lines_df.set_index( 'modifies_line' )[ wanted_cols ]

In [273]:
modifiers_lines_df['modifier_type'].value_counts()

VALUE         8763
OCCURS        4056
REDEFINES     1090
INDEXED BY      20
BLANK ON         3
OLQ              2
Name: modifier_type, dtype: int64

In [274]:
duplicate_modifiers = modifiers_lines_df[ modifiers_lines_df.index.duplicated(keep=False) ].sort_index()

In [275]:
duplicate_counts = duplicate_modifiers.groupby( 'modifies_line' ).apply( lambda group_df: group_df[ "modifier_type" ].value_counts() )

In [276]:
pd.set_option( 'display.max_rows', 100 )
#pd.set_option( 'display.max_rows', 100 )

In [277]:
#duplicate_counts

In [278]:
modifiers_lines_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13934 entries, 57 to 171635
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   modifier_type   13934 non-null  object
 1   modifier_value  13934 non-null  object
dtypes: object(2)
memory usage: 842.6+ KB


In [279]:
#modifiers_lines_df

In [280]:
pivoted_modifiers = modifiers_lines_df.pivot_table(
    values='modifier_value',
    index='modifies_line',
    columns='modifier_type',
    # Either populate the cell with the single value, or listify multiple values:
    aggfunc=lambda v: list( v ) if len( v ) > 1 else v,
    fill_value=""
)

In [281]:
pivoted_modifiers = pivoted_modifiers.reset_index().rename( columns={'modifies_line': 'line' } )

In [282]:
pivoted_modifiers.shape

(13080, 7)

In [283]:
all_fields_info

,line,table_name,table_index,table_vers,table_created_date,table_updated_date,field_name,data_type,indent_number,indent_space_count,declaration_step,end,raw_field_name
0,5,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CTY-CDE-FMHA,,05,3,100,DISPLAY,ST-CTY-CDE-FMHA-4541
1,6,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CDE-FMHA,9(2),10,4,200,DISPLAY,ST-CDE-FMHA-4541
2,7,ABSDOMN,0,1,1987-09-29,1987-10-15,CTY-CDE-FMHA,9(3),10,4,300,DISPLAY,CTY-CDE-FMHA-4541
3,8,ABSDOMN,0,1,1987-09-29,1987-10-15,NME-CTY,X(20),05,3,400,DISPLAY,NME-CTY-4541
4,9,ABSDOMN,0,1,1987-09-29,1987-10-15,LST-UPDTE-DTE,9(06),05,3,500,DISPLAY,LST-UPDTE-DTE-4541
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89283,171637,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,RACE-DTRMNTN-CDE,X,05,3,5100,DISPLAY,W502-RACE-DTRMNTN-CDE
89284,171638,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,ST-CDE,9(2),05,3,5200,DISPLAY,W502-ST-CDE
89285,171639,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,FILLER,X(0002),05,3,5300,DISPLAY,FILLER
89286,171640,502-TEMP-DTL-RCRD,3660,4,2002-03-12,2002-03-14,CR-REPT-FEE-CDE,9(1),05,3,5400,DISPLAY,W502-CR-REPT-FEE-CDE


In [284]:
all_fields_info = all_fields_info.merge( pivoted_modifiers, on='line', how='left' )

In [285]:
all_fields_info.drop(columns=['line']).to_excel( '2022-12-08_PLAS_IDMS_data_structure.xlsx' )